# Sampler for the School Example

In [14]:
# Import PyMC3
from pymc3 import  *

# -- Load libraries --
# Data manipulation libraries
import numpy as np
from numpy.linalg import inv 
import scipy as sp
import pandas as pd

# Plotting libaries
import plotnine as pn
from plotnine import *
import seaborn as sns
import matplotlib.pyplot as plt
import utils


# Supressing warnings
import warnings
warnings.filterwarnings("ignore")
# warnings.filterwarnings("default")

# Use a theano shared variable to be able to exchange the data the model runs on
from theano import shared

In [15]:
import pandas as pd
mu = pd.Series([28,  8, -3,  7, -1,  1, 18, 12]) 
sigma = pd.Series([15, 10, 16, 11,  9, 11, 10, 18])


In [17]:
# For expressiveness we include the full model here
# Instantiate base model
model = Model()
niter = 5000

with model:
    # Define hyperpriors
    tau2_0 = InverseGamma('tau2_0', alpha=1,beta=1, testval=1.)
    mu_0 = Normal('mu0', 0, sigma=30)

    # Define mean
    mu_par = Normal('mu_par', mu=mu_0, sigma=tau2_0, shape=8)

    # Define likelihood
    likelihood = Normal('y_like', mu = mu_par, sigma=sigma, observed=mu)

    # Inference!
    # inference
    start = find_MAP() # Use MAP estimate (optimization) as the initial state for MCMC
    step = NUTS() # Have a choice of samplers
    trace = sample(niter, step, start, random_seed=123, progressbar=True, cores=2)

# -- Summary stats ---/
summary(trace).round(2)

logp = -29.725, ||grad|| = 12.043: 100%|██████████| 31/31 [00:00<00:00, 3230.48it/s]   
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu_par, mu0, tau2_0]
Sampling 2 chains, 822 divergences: 100%|██████████| 11000/11000 [00:10<00:00, 1062.90draws/s]
There were 428 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6391337981751473, but should be close to 0.8. Try to increase the number of tuning steps.
There were 394 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6416675956345369, but should be close to 0.8. Try to increase the number of tuning steps.
The estimated number of effective samples is smaller than 200 for some parameters.


,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
mu0,7.70,3.90,-1.01,14.66,0.22,0.16,312.0,312.0,325.0,282.0,1.01
mu_par[0],8.35,4.76,-0.79,17.52,0.22,0.16,466.0,466.0,423.0,357.0,1.01
mu_par[1],7.71,4.37,-1.05,16.18,0.22,0.16,380.0,380.0,387.0,402.0,1.01
mu_par[2],7.38,4.64,-1.37,16.29,0.22,0.16,429.0,429.0,411.0,913.0,1.01
mu_par[3],7.67,4.39,-1.00,16.28,0.22,0.16,393.0,393.0,399.0,424.0,1.01
mu_par[4],7.12,4.42,-1.85,15.14,0.22,0.16,392.0,392.0,395.0,809.0,1.01
mu_par[5],7.38,4.47,-1.54,15.64,0.23,0.16,385.0,385.0,385.0,921.0,1.01
mu_par[6],8.33,4.46,-0.25,17.46,0.22,0.15,428.0,428.0,415.0,357.0,1.01
mu_par[7],7.80,4.64,-1.38,16.85,0.23,0.16,423.0,423.0,417.0,380.0,1.01
tau2_0,2.08,2.08,0.35,5.80,0.10,0.07,459.0,459.0,110.0,115.0,1.03


- The model here strongly regularizes for the inverse gamma
- If we allow for a less strict regularization

In [19]:
# For expressiveness we include the full model here
# Instantiate base model
model2 = Model()
niter = 5000

with model2:
    # Define hyperpriors
    tau2_0 = HalfCauchy('tau2_0', 100, testval=1.)
    mu_0 = Normal('mu0', 0, sigma=30)

    # Define mean
    mu_par = Normal('mu_par', mu=mu_0, sigma=tau2_0, shape=8)

    # Define likelihood
    likelihood = Normal('y_like', mu = mu_par, sigma=sigma, observed=mu)

    # Inference!
    # inference
    start = find_MAP() # Use MAP estimate (optimization) as the initial state for MCMC
    step = NUTS() # Have a choice of samplers
    trace2 = sample(niter, step, start, random_seed=123, progressbar=True, cores=2)

# -- Summary stats ---/
summary(trace2).round(2)

logp = 83.412, ||grad|| = 1.918e+07: 100%|██████████| 126/126 [00:00<00:00, 3433.42it/s]   
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu_par, mu0, tau2_0]
Sampling 2 chains, 281 divergences: 100%|██████████| 11000/11000 [00:07<00:00, 1459.15draws/s]
There were 138 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6764449424418525, but should be close to 0.8. Try to increase the number of tuning steps.
There were 143 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6378763413001113, but should be close to 0.8. Try to increase the number of tuning steps.
The estimated number of effective samples is smaller than 200 for some parameters.


,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
mu0,7.65,4.99,-1.54,17.10,0.13,0.09,1487.0,1487.0,1402.0,3390.0,1.01
mu_par[0],11.27,8.21,-2.15,28.22,0.20,0.14,1604.0,1604.0,1316.0,4250.0,1.00
mu_par[1],7.74,6.25,-4.05,19.74,0.12,0.10,2489.0,2098.0,2263.0,4409.0,1.00
mu_par[2],5.92,7.51,-8.86,20.36,0.15,0.11,2553.0,2367.0,2261.0,4297.0,1.00
mu_par[3],7.53,6.58,-4.35,20.94,0.12,0.09,2766.0,2528.0,2416.0,5221.0,1.00
mu_par[4],4.89,6.22,-7.22,16.24,0.15,0.11,1774.0,1555.0,1663.0,3144.0,1.00
mu_par[5],5.91,6.62,-7.34,17.80,0.14,0.10,2207.0,2161.0,2063.0,4444.0,1.00
mu_par[6],10.54,6.78,-1.64,23.80,0.20,0.14,1174.0,1174.0,1035.0,1795.0,1.00
mu_par[7],8.26,7.86,-5.90,24.30,0.15,0.12,2891.0,2159.0,2425.0,4106.0,1.00
tau2_0,6.74,5.16,1.24,15.71,0.24,0.17,472.0,472.0,168.0,101.0,1.00


In [ ]:
summary(trace2).round(2)